In [2]:
from fastai import *
import matplotlib.pyplot as plt
from fastai.vision import *
from fastai.callbacks.hooks import num_features_model
import cv2

In [3]:
from fastai.callbacks.hooks import num_features_model
from fastai.vision import create_head

In [4]:
from motion.detect_human import BBoxDataset

In [5]:
SZ = 500
bbox_ds = BBoxDataset("coco/val2017_one_human.csv",type = 'val', size = SZ)
bbox_ds_val =  BBoxDataset("coco/val2017_one_human_val.csv",type='valid', size = SZ)
print(bbox_ds_val[0][0].shape,bbox_ds_val[0][1].shape)

Dataset has 1150 samples.
Dataset has 225 samples.
(3, 500, 500) (4,)


In [6]:
BS = 8

train_dl = DataLoader(bbox_ds, BS)
valid_dl = DataLoader(bbox_ds_val, BS, 
                      shuffle=False, num_workers=2, pin_memory=True)

data_bunch = DataBunch(train_dl, valid_dl)

In [7]:
class FlukeDetector2(nn.Module):
    def __init__(self, arch=models.resnet18):
        super().__init__() 
        self.cnn = create_body(arch)
        self.head = create_head(num_features_model(self.cnn) * 2, 4)
        
    def forward(self, im):
        x = self.cnn(im)
        x = self.head(x)
        return x.sigmoid_()

## Without loss function

In [8]:
learn = Learner(data_bunch, FlukeDetector2(arch=models.resnet50))
learn = learn.load('fastai_bbox_detect_humans_val3')

In [9]:
preds, targs = learn.get_preds()

In [12]:
predicted_bboxes = ((preds) ).numpy()
targets = ((targs) ).numpy()
predicted_bboxes[0]

array([1.345302, 1.258236, 2.051251, 2.114838], dtype=float32)

Note the result is above 1 while the model end with a sigmoid function.

## With loss function

In [13]:
from torch.nn import L1Loss
learn = Learner(data_bunch, FlukeDetector2(arch=models.resnet50), loss_func=L1Loss())
learn = learn.load('fastai_bbox_detect_humans_val3')

In [14]:
preds, targs = learn.get_preds()

In [15]:
predicted_bboxes = ((preds) ).numpy()
targets = ((targs) ).numpy()
predicted_bboxes[0]

array([0.296618, 0.229711, 0.71845 , 0.748978], dtype=float32)

Different results! 

In [17]:
targets[0]

array([0.548, 0.236, 0.802, 0.868], dtype=float32)

and this time they seem fine what I expected.